In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("s8a-dataframes-api").getOrCreate()

# Lectura de CSV con el ; como separador de columnas y con encabezado
df = spark.read.option("delimiter",";").option("header", "true").csv("pdi_sales_small.csv")
    
df.printSchema()

22/03/11 15:50:29 WARN Utils: Your hostname, MacBook-Air-de-Aitor.local resolves to a loopback address: 127.0.0.1; using 192.168.1.203 instead (on interface en0)
22/03/11 15:50:29 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/03/11 15:50:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


root
 |-- ProductID: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Zip: string (nullable = true)
 |-- Units: string (nullable = true)
 |-- Revenue: string (nullable = true)
 |-- Country: string (nullable = true)



In [2]:
df.show()

+---------+---------+---------------+-----+-------+-------+
|ProductID|     Date|            Zip|Units|Revenue|Country|
+---------+---------+---------------+-----+-------+-------+
|      725|1/15/1999|41540          |    1|  115.5|Germany|
|      787| 6/6/2002|41540          |    1|  314.9|Germany|
|      788| 6/6/2002|41540          |    1|  314.9|Germany|
|      940|1/15/1999|22587          |    1|  687.7|Germany|
|      396|1/15/1999|22587          |    1|  857.1|Germany|
|      734|4/10/2003|22587          |    1|  330.7|Germany|
|      769|2/15/1999|22587          |    1|  257.2|Germany|
|      499|1/15/1999|12555          |    1|  846.3|Germany|
|     2254|1/15/1999|40217          |    1|   57.7|Germany|
|       31|5/31/2002|40217          |    1|  761.2|Germany|
|      475|2/15/1999|13583          |    1|  970.2|Germany|
|      510|1/15/1999|22337          |    1|  837.1|Germany|
|      499| 6/5/2002|22337          |    1|    883|Germany|
|      289|2/15/1999|13587          |   

In [3]:
from pyspark.sql.functions import to_date
df = df.withColumn("Units", df.Units.cast("int")).withColumn("Revenue", df.Revenue.cast("double"))
df = df.withColumn("Date", to_date(df.Date, "M/d/yyy"))
df.printSchema()

root
 |-- ProductID: string (nullable = true)
 |-- Date: date (nullable = true)
 |-- Zip: string (nullable = true)
 |-- Units: integer (nullable = true)
 |-- Revenue: double (nullable = true)
 |-- Country: string (nullable = true)



In [4]:
df.show(5)

+---------+----------+---------------+-----+-------+-------+
|ProductID|      Date|            Zip|Units|Revenue|Country|
+---------+----------+---------------+-----+-------+-------+
|      725|1999-01-15|41540          |    1|  115.5|Germany|
|      787|2002-06-06|41540          |    1|  314.9|Germany|
|      788|2002-06-06|41540          |    1|  314.9|Germany|
|      940|1999-01-15|22587          |    1|  687.7|Germany|
|      396|1999-01-15|22587          |    1|  857.1|Germany|
+---------+----------+---------------+-----+-------+-------+
only showing top 5 rows



In [5]:
df.select("ProductID","Revenue").show(3)

+---------+-------+
|ProductID|Revenue|
+---------+-------+
|      725|  115.5|
|      787|  314.9|
|      788|  314.9|
+---------+-------+
only showing top 3 rows



In [6]:
df.select(df.ProductID,(df.Revenue+10).alias("VentasMas10")).show(3)

+---------+-----------+
|ProductID|VentasMas10|
+---------+-----------+
|      725|      125.5|
|      787|      324.9|
|      788|      324.9|
+---------+-----------+
only showing top 3 rows



In [7]:
df.sort("Revenue", descending=False).show(5)

+---------+----------+---------------+-----+-------+-------+
|ProductID|      Date|            Zip|Units|Revenue|Country|
+---------+----------+---------------+-----+-------+-------+
|     2314|1999-05-15|46045          |    1|   13.9|Germany|
|     1974|1999-03-15|R3H            |    1|   52.4|Canada |
|     1975|1999-01-15|R3S            |    1|   52.4|Canada |
|     1974|1999-04-15|R3H            |    1|   52.4|Canada |
|     1974|1999-03-15|R3B            |    1|   52.4|Canada |
+---------+----------+---------------+-----+-------+-------+
only showing top 5 rows



In [8]:
df.select("ProductID","Revenue").sort("Revenue").show(5)

+---------+-------+
|ProductID|Revenue|
+---------+-------+
|     2314|   13.9|
|     1974|   52.4|
|     1975|   52.4|
|     1974|   52.4|
|     1974|   52.4|
+---------+-------+
only showing top 5 rows



In [9]:
df.sort(df.Revenue.desc()).show(5)

+---------+----------+---------------+-----+-------+-------+
|ProductID|      Date|            Zip|Units|Revenue|Country|
+---------+----------+---------------+-----+-------+-------+
|      495|1999-03-15|75213 CEDEX 16 |   77|43194.1|France |
|      495|2000-03-01|75391 CEDEX 08 |   18|10395.0|France |
|      464|2003-06-11|75213 CEDEX 16 |   16|10075.8|France |
|      464|2000-08-01|22397          |   17| 9817.5|Germany|
|      495|2000-03-01|06175 CEDEX 2  |   16| 9240.0|France |
+---------+----------+---------------+-----+-------+-------+
only showing top 5 rows



In [10]:
df.sort(["Revenue","Units"], ascending=[0,1]).show(5)

+---------+----------+---------------+-----+-------+-------+
|ProductID|      Date|            Zip|Units|Revenue|Country|
+---------+----------+---------------+-----+-------+-------+
|      495|1999-03-15|75213 CEDEX 16 |   77|43194.1|France |
|      495|2000-03-01|75391 CEDEX 08 |   18|10395.0|France |
|      464|2003-06-11|75213 CEDEX 16 |   16|10075.8|France |
|      464|2000-08-01|22397          |   17| 9817.5|Germany|
|      495|2000-03-01|06175 CEDEX 2  |   16| 9240.0|France |
+---------+----------+---------------+-----+-------+-------+
only showing top 5 rows



In [11]:
df.groupBy("Country").count().show()

+-------+-----+
|Country|count|
+-------+-----+
|Germany|30059|
| France|    1|
|France |30059|
|Canada |30060|
|Mexico |30060|
+-------+-----+



In [12]:
# Corregimos el error de " France"
from pyspark.sql.functions import trim
df = df.withColumn("Country", trim(df.Country))
df.groupBy("Country").count().show()

+-------+-----+
|Country|count|
+-------+-----+
|Germany|30059|
| France|30060|
| Mexico|30060|
| Canada|30060|
+-------+-----+



In [13]:
df.groupBy("Country").avg("Revenue").show()

+-------+------------------+
|Country|      avg(Revenue)|
+-------+------------------+
|Germany| 498.4237665923521|
| France| 402.1271490352905|
| Mexico| 379.0618330007039|
| Canada|387.31251497012323|
+-------+------------------+



In [14]:
df.groupBy("Country").sum("Units").show()

+-------+----------+
|Country|sum(Units)|
+-------+----------+
|Germany|     31746|
| France|     31739|
| Mexico|     31095|
| Canada|     31148|
+-------+----------+



In [54]:
# Varuas agregaciones a la vez
df.groupBy("Country").agg({"Units":"sum", "Revenue":"avg"}).show()

+-------+------------------+----------+
|Country|      avg(Revenue)|sum(Units)|
+-------+------------------+----------+
|Germany| 498.4237665923521|     31746|
| France| 402.1271490352905|     31739|
| Mexico| 379.0618330007039|     31095|
| Canada|387.31251497012323|     31148|
+-------+------------------+----------+



In [56]:
#Filter
dfMas40 = df.filter(df.Units > 40)
dfMas40.show()

+---------+----------+---------------+-----+-------+-------+
|ProductID|      Date|            Zip|Units|Revenue|Country|
+---------+----------+---------------+-----+-------+-------+
|      495|1999-03-15|75213 CEDEX 16 |   77|43194.1| France|
|     2091|1999-06-15|40213          |   41| 6240.1|Germany|
|     2091|1999-10-15|40213          |   41| 6347.7|Germany|
+---------+----------+---------------+-----+-------+-------+



In [48]:
df.filter(df.Date == "2000-03-01").show()

+---------+----------+---------------+-----+-------+-------+
|ProductID|      Date|            Zip|Units|Revenue|Country|
+---------+----------+---------------+-----+-------+-------+
|      396|2000-03-01|H1B            |    1|  865.7| Canada|
|      413|2000-03-01|R3G            |    1| 1181.0| Canada|
|      413|2000-03-01|M5N            |    1| 1181.0| Canada|
|      475|2000-03-01|M5R            |    1|  978.1| Canada|
|      606|2000-03-01|M4V            |    1|  314.7| Canada|
|      663|2000-03-01|H1B            |    1|  771.8| Canada|
|      681|2000-03-01|L5N            |    1|  648.6| Canada|
|      413|2000-03-01|L5V            |    1| 1181.0| Canada|
|      513|2000-03-01|L5N            |    1|  451.4| Canada|
|      574|2000-03-01|R3H            |    1|  839.7| Canada|
|      596|2000-03-01|H1B            |    1|  752.3| Canada|
|      604|2000-03-01|M5X            |    1|  461.7| Canada|
|      609|2000-03-01|M5S            |    1|  802.2| Canada|
|      559|2000-03-01|R3

In [49]:
df.filter((df.Date == "2000-03-01") & (df.Units > 10)).show()

+---------+----------+---------------+-----+-------+-------+
|ProductID|      Date|            Zip|Units|Revenue|Country|
+---------+----------+---------------+-----+-------+-------+
|      495|2000-03-01|75391 CEDEX 08 |   18|10395.0| France|
|      495|2000-03-01|06175 CEDEX 2  |   16| 9240.0| France|
+---------+----------+---------------+-----+-------+-------+



In [50]:
df.filter((df.Country == "Germany") | (df.Country == "France")).show()

+---------+----------+---------------+-----+-------+-------+
|ProductID|      Date|            Zip|Units|Revenue|Country|
+---------+----------+---------------+-----+-------+-------+
|      725|1999-01-15|41540          |    1|  115.5|Germany|
|      787|2002-06-06|41540          |    1|  314.9|Germany|
|      788|2002-06-06|41540          |    1|  314.9|Germany|
|      940|1999-01-15|22587          |    1|  687.7|Germany|
|      396|1999-01-15|22587          |    1|  857.1|Germany|
|      734|2003-04-10|22587          |    1|  330.7|Germany|
|      769|1999-02-15|22587          |    1|  257.2|Germany|
|      499|1999-01-15|12555          |    1|  846.3|Germany|
|     2254|1999-01-15|40217          |    1|   57.7|Germany|
|       31|2002-05-31|40217          |    1|  761.2|Germany|
|      475|1999-02-15|13583          |    1|  970.2|Germany|
|      510|1999-01-15|22337          |    1|  837.1|Germany|
|      499|2002-06-05|22337          |    1|  883.0|Germany|
|      289|1999-02-15|13

## Conjuntos

In [51]:
dfFrance = df.filter(df.Country == "France")
dfGermany = df.filter(df.Country == "Germany")
dfFB = dfFrance.unionAll(dfGermany)
dfFB.select("Country").distinct().show()

+-------+
|Country|
+-------+
|Germany|
| France|
+-------+



In [52]:
dfSinFrance = df.exceptAll(dfFrance)
dfSinFrance.select("Country").distinct().show()

+-------+
|Country|
+-------+
|Germany|
| Mexico|
| Canada|
+-------+



In [58]:
dfMas40France = dfMas40.intersect(dfFrance)
dfMas40France.show()

+---------+----------+---------------+-----+-------+-------+
|ProductID|      Date|            Zip|Units|Revenue|Country|
+---------+----------+---------------+-----+-------+-------+
|      495|1999-03-15|75213 CEDEX 16 |   77|43194.1| France|
+---------+----------+---------------+-----+-------+-------+



## Reduciendo

In [60]:
# Nos quedamos solo con 10
dfPrimeros10 = df.limit(10)
dfPrimeros10.show()

+---------+----------+---------------+-----+-------+-------+
|ProductID|      Date|            Zip|Units|Revenue|Country|
+---------+----------+---------------+-----+-------+-------+
|      725|1999-01-15|41540          |    1|  115.5|Germany|
|      787|2002-06-06|41540          |    1|  314.9|Germany|
|      788|2002-06-06|41540          |    1|  314.9|Germany|
|      940|1999-01-15|22587          |    1|  687.7|Germany|
|      396|1999-01-15|22587          |    1|  857.1|Germany|
|      734|2003-04-10|22587          |    1|  330.7|Germany|
|      769|1999-02-15|22587          |    1|  257.2|Germany|
|      499|1999-01-15|12555          |    1|  846.3|Germany|
|     2254|1999-01-15|40217          |    1|   57.7|Germany|
|       31|2002-05-31|40217          |    1|  761.2|Germany|
+---------+----------+---------------+-----+-------+-------+



In [62]:
dfPrimeros10sinZip = dfPrimeros10.drop("Zip")
dfPrimeros10sinZip.show()

+---------+----------+-----+-------+-------+
|ProductID|      Date|Units|Revenue|Country|
+---------+----------+-----+-------+-------+
|      725|1999-01-15|    1|  115.5|Germany|
|      787|2002-06-06|    1|  314.9|Germany|
|      788|2002-06-06|    1|  314.9|Germany|
|      940|1999-01-15|    1|  687.7|Germany|
|      396|1999-01-15|    1|  857.1|Germany|
|      734|2003-04-10|    1|  330.7|Germany|
|      769|1999-02-15|    1|  257.2|Germany|
|      499|1999-01-15|    1|  846.3|Germany|
|     2254|1999-01-15|    1|   57.7|Germany|
|       31|2002-05-31|    1|  761.2|Germany|
+---------+----------+-----+-------+-------+



In [66]:
df.count()

120239

In [67]:
dfSinDuplicados = df.dropDuplicates()
df.count()

120239

In [71]:
dfSinProductosRepetidos = df.dropDuplicates(["ProductID"])
dfSinProductosRepetidos.count()

799